### Download data

In [4]:
from datasets import load_dataset

ds_train = load_dataset("re_dial", revision="refs/convert/parquet", split="train")
ds_test = load_dataset("re_dial", revision="refs/convert/parquet", split="test")

ds_train.to_pandas().to_parquet("data/raw/train.parquet")
ds_test.to_pandas().to_parquet("data/raw/test.parquet")

c:\Users\Matheus\Documents\Git\GPT-projeto-crs-PT-BR\venv_crs\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 145.18it/s]
Generating train split: 10006 examples [00:00, 239908.80 examples/s]
Generating test split: 1342 examples [00:00, 170372.18 examples/s]


### Verify Interim

In [ ]:
import pandas as pd

interim_path = 'data/processed/translated_ds_011.parquet'

df = pd.read_parquet(interim_path)
print(f'total de linhas: {len(df)}')

# non_translated_count = df['text_translated'].isna().sum()
# translated_count = len(df) - non_translated_count

# print(f'Total de mensagens traduzidas: {translated_count}')
# print(f'Total de mensagens não traduzidas: {non_translated_count}')


### Split Dataset

In [ ]:
import pandas as pd
import numpy as np

# Carregando o dataset

PATH = 'data/raw/ds.parquet'

df = pd.read_parquet(PATH)

# Dividindo o dataframe em 10 partes
splits = np.array_split(df, 10)

# Salvando cada parte como um arquivo parquet separado
for idx, split in enumerate(splits, 1):
    filename = f'data/raw/ds_{idx:03}.parquet'
    split.to_parquet(filename, index=False)
    print(f"Saved {filename}")


### Example generating batchs samples

In [19]:
import os
import re
import pandas as pd

PATH = 'data/processed'
df_train = []
all_movies = []

df = []

for filename in os.listdir(PATH):

    if filename.endswith('.parquet') and filename != 'translated_ds_011.parquet':
        df.append(pd.read_parquet(os.path.join(PATH, filename)))

df_train = pd.concat(df).reset_index(drop=True)
df_test = pd.read_parquet('data/processed/translated_ds_011.parquet')

In [27]:
all_movies = [j['movieName'] for i in df_train['movieMentions'].tolist() for j in i]
all_movies.extend([j['movieName'] for i in df_test['movieMentions'].tolist() for j in i])

In [33]:
# upload df_train and df_test to huggingface datasets
from datasets import Dataset

ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

# ds_train.save_to_disk('data/processed/ds_train')
# ds_test.save_to_disk('data/processed/ds_test')


In [35]:
ds_train.push_to_hub("matheusrdgsf/re_dial_ptbr", split="train")
ds_test.push_to_hub("matheusrdgsf/re_dial_ptbr", split="test")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [45]:
import json
json.dumps(example, ensure_ascii=False)

'{"conversationId": 391, "messages": [{"messageId": 1021, "senderWorkerId": 0, "text": "Hi there, how are you? I\'m looking for movie recommendations", "timeOffset": 0}, {"messageId": 1022, "senderWorkerId": 1, "text": "I am doing okay. What kind of movies do you like?", "timeOffset": 15}, {"messageId": 1023, "senderWorkerId": 0, "text": "I like animations like @84779 and @191602", "timeOffset": 66}, {"messageId": 1024, "senderWorkerId": 0, "text": "I also enjoy @122159", "timeOffset": 86}, {"messageId": 1025, "senderWorkerId": 0, "text": "Anything artistic", "timeOffset": 95}, {"messageId": 1026, "senderWorkerId": 1, "text": "You might like @165710 that was a good movie.", "timeOffset": 135}, {"messageId": 1027, "senderWorkerId": 0, "text": "What\'s it about?", "timeOffset": 151}, {"messageId": 1028, "senderWorkerId": 1, "text": "It has Alec Baldwin it is about a baby that works for a company and gets adopted it is very funny", "timeOffset": 207}, {"messageId": 1029, "senderWorkerId":

In [36]:
def id2text(text, movies):
    for movie_id in re.findall(r'@\d+', text):
        if movies[movies['movieId'] == movie_id[1:]].empty:
            movie_name = '<unk>'
        else:
            movie_name = movies[movies['movieId'] == movie_id[1:]]['movieName'].iloc[0]
        text = text.replace(movie_id, movie_name)
    return text

In [10]:
movies = pd.DataFrame(df['movieMentions'].explode().drop_duplicates().dropna().reset_index(drop=True).tolist())

### V1 Dataset

In [11]:
#from string import punctuation
#from tqdm import tqdm

#dict_punctuation = {i: j for j, i in enumerate(punctuation)}

#df_train = []
# df = pd.read_parquet('data/processed/translated_ds_011.parquet')

# for _, row in tqdm(df.iterrows(), total=df.shape[0]):
#     df_explode = pd.DataFrame(row[['messages_translated']].explode().tolist())
#     # print(df_explode)
#     df_explode['text'] = df_explode.apply(lambda x: id2text(x['text'], movies), axis=1)
#     #print(df_explode.iloc[0])
#     worker_id = df_explode.iloc[0]['senderWorkerId']
#     instruction = ''
#     response = ''

#     changed = False

#     for index, message in df_explode.iterrows():
        
#         if changed == False:
#             if message['senderWorkerId'] == worker_id:
#                 instruction += message['text']
#                 if instruction[-1] not in dict_punctuation:
#                     instruction+='.'
#             else:
#                 changed = True
#                 response += message['text']
#                 if response[-1] not in dict_punctuation:
#                     response+='.'
#         else:
#             if message['senderWorkerId'] != worker_id:
#                 response += message['text']
#                 if response[-1] not in dict_punctuation:
#                     response+='.'
#             else:
#                 changed = False
#                 df_train.append({'initiator': instruction, 'respondant': response})
#                 response = ''
#                 instruction = message['text']
#                 if instruction[-1] not in dict_punctuation:
#                     instruction+='.'
# df_train = pd.DataFrame(df_train)

# def generate_sample(conversation):
#         return "<|system|>\n Você é um chatbot de recomendação de filmes, converse com o usuário para indicar filmes apropriados.</s>\n<|user|>\n" + conversation['initiator'] + "</s>\n<|assistant|>\n" + conversation['respondant'] + "</s>\n"

# df_train['sample'] = df_train.apply(lambda x: generate_sample(x), axis=1)
# df_train.drop(['initiator', 'respondant'], axis=1, inplace=True)

# df_train.to_parquet('data/processed/test.parquet', index=False)

100%|██████████| 9005/9005 [01:10<00:00, 126.91it/s]


### V2 Dataset

In [46]:
tokenizer.chat_template

"{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

In [53]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")

In [54]:
messages = [
    {"role": "system", "content": "Você é um chatbot para indicação de filmes. Responda de maneira educada sugestões de filmes para os usuários."},
    {"role": "user", "content": "Hi there!"},
    {"role": "assistant", "content": "Nice to meet you!"},
    {"role": "user", "content": "Can I ask a question?"}
]

tokenizer.apply_chat_template(messages, tokenize=False)

'<|system|>\nVocê é um chatbot para indicação de filmes. Responda de maneira educada sugestões de filmes para os usuários.</s>\n<|user|>\nHi there!</s>\n<|assistant|>\nNice to meet you!</s>\n<|user|>\nCan I ask a question?</s>\n'

In [39]:
from transformers import AutoTokenizer
from string import punctuation
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")

def process_dataset(df, movies):
    
    dataset = []

    for _, row in tqdm(df.iterrows(), total=df.shape[0]):
        df_explode = pd.DataFrame(row[['messages_translated']].explode().tolist())
        df_explode['text'] = df_explode.apply(lambda x: id2text(x['text'], movies), axis=1)
        worker_id = df_explode.iloc[0]['senderWorkerId']

        message_template = [{"role": "system", "content": "Você é um chatbot para indicação de filmes. Responda de maneira educada sugestões de filmes para os usuários."}]

        for index, message in df_explode.iterrows():
            
            if message['senderWorkerId'] == worker_id:
                message_template.append({"role": "user", "content": message['text']})
            else:
                message_template.append({"role": "assistant", "content": message['text']})

        dataset.append(tokenizer.apply_chat_template(message_template, tokenize=False))
    
    return dataset

# Train
df_train = process_dataset(df, movies)

# Test
df_test = pd.read_parquet('data/processed/translated_ds_011.parquet')
movies_test = pd.DataFrame(df_test['movieMentions'].explode().drop_duplicates().dropna().reset_index(drop=True).tolist())
df_test = process_dataset(pd.read_parquet('data/processed/translated_ds_011.parquet'), movies_test)

100%|██████████| 1342/1342 [00:08<00:00, 166.81it/s]


In [44]:
pd.DataFrame(df_train, columns=['sample']).to_parquet('data/processed/colab/v2/train.parquet', index=False)
pd.DataFrame(df_test, columns=['sample']).to_parquet('data/processed/colab/v2/test.parquet', index=False)

# Save dataset to hub

In [ ]:
pd.DataFrame(df_train, columns=['sample']).to_parquet('data/processed/colab/v2/train.parquet', index=False)
pd.DataFrame(df_test, columns=['sample']).to_parquet('data/processed/colab/v2/test.parquet', index=False)

In [2]:
import pandas as pd

df_train = pd.read_parquet('data/processed/colab/v2/train.parquet')

In [4]:
df_train['sample'].iloc[0]

'<|system|>\nVocê é um chatbot para indicação de filmes. Responda de maneira educada sugestões de filmes para os usuários.</s>\n<|user|>\nOlá, como você está? Estou procurando recomendações de filmes.</s>\n<|assistant|>\nEu estou indo bem. Qual tipo de filmes você gosta?</s>\n<|user|>\nEu gosto de animações como The Triplets of Belleville (2003) e Waking Life (2001).</s>\n<|user|>\nEu também gosto de Mary and Max (2009).</s>\n<|user|>\nQualquer coisa artística</s>\n<|assistant|>\nVocê pode gostar de saber que foi um bom filme.</s>\n<|user|>\nO que é isso?</s>\n<|assistant|>\nTem um bebê que trabalha para uma empresa e é adotado. É muito engraçado.</s>\n<|user|>\nIsso parece ser uma comédia legal.</s>\n<|user|>\nVocê tem alguma recomendação animada que seja um pouco mais dramática, como por exemplo A Scanner Darkly  (2006)?</s>\n<|user|>\nEu gosto de comédias, mas prefiro filmes com um pouco mais de profundidade.</s>\n<|assistant|>\nIsso é um desafio, mas eu me lembrarei de algo.</s>\n<

### Evaluation

In [8]:
import re
from fuzzywuzzy import fuzz

def regex_movies(text):
  text = text.replace('"', '')
  regex = r'\b[A-Z][a-zA-Z]+(?:\s[A-Z][A-Za-z]+)*\s\(\d{4}\)'
  movies = re.findall(regex, text)

  return list(set(movies))

def get_similarity(text_a, text_b):
  similarity = fuzz.ratio(text_a, text_b)
  return similarity

def compute_metrics(data, threshold=80):

  expected_movies = data['expected_movies']
  #print(expected_movies)

  # Has Movie
  pred_movies = data['pred_movies']
  #print(pred_movies)

  # Hits
  hits = 0
  for pred_movie in pred_movies:
    for expected_movie in expected_movies:
      if fuzz.ratio(pred_movie, expected_movie) > threshold:
        # print(fuzz.ratio(pred_movie, expected_movie))
        hits+=1

  # salvar csv com data_eval['context'], data_eval['response'], completion, inference_tinme, has_movie, hits


  return {
      'has_movie': 1 if pred_movies else 0,
      'hits': hits
  }



c:\Users\Matheus\miniconda3\envs\dl_venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [9]:
import pandas as pd

cesar = pd.read_parquet('data/results/test_8B_ours_generated_3.parquet')
gptq = pd.read_parquet('data/results/test_gptq_8b_sf_generated.parquet')
base = pd.read_parquet('data/results/test_base_sf_generated.parquet')
# pd.DataFrame(all_movies, columns=['movieName']).to_csv('data/results/all_movies.csv', index=False)
all_movies = pd.read_csv('data/results/all_movies.csv')

cesar['pred_movies'] = cesar['generated_response'].apply(lambda x: regex_movies(x))
gptq['pred_movies'] = gptq['generated_response'].apply(lambda x: regex_movies(x))
base['pred_movies'] = base['generated_response'].apply(lambda x: regex_movies(x))


In [29]:
cesar[['has_movie', 'hits']] = cesar.apply(lambda x: compute_metrics(x), axis=1, result_type='expand')
gptq[['has_movie', 'hits']] = gptq.apply(lambda x: compute_metrics(x), axis=1, result_type='expand')
base[['has_movie', 'hits']] = base.apply(lambda x: compute_metrics(x), axis=1, result_type='expand')

In [33]:
print(cesar.value_counts('has_movie', normalize=True))
print(cesar.value_counts('hits', normalize=True))

has_movie
1    0.851351
0    0.148649
dtype: float64
hits
0    0.905405
1    0.081081
2    0.013514
dtype: float64


In [31]:
print(gptq.value_counts('has_movie', normalize=True))
print(gptq.value_counts('hits', normalize=True))

has_movie
1    0.783784
0    0.216216
dtype: float64
hits
0    0.851351
1    0.135135
2    0.013514
dtype: float64


In [32]:
print(base.value_counts('has_movie', normalize=True))
print(base.value_counts('hits', normalize=True))

has_movie
1    0.783784
0    0.216216
dtype: float64
hits
0    0.837838
1    0.148649
2    0.013514
dtype: float64


In [10]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

cesar['language'] = cesar['generated_response'].apply(lambda x: pipe(x))
gptq['language'] = gptq['generated_response'].apply(lambda x: pipe(x))
base['language'] = base['generated_response'].apply(lambda x: pipe(x))

In [19]:
cesar['majority_language'] = cesar['language'].apply(lambda x: x[0]['label'])
gptq['majority_language'] = gptq['language'].apply(lambda x: x[0]['label'])
base['majority_language'] = base['language'].apply(lambda x: x[0]['label'])

In [22]:
print(cesar['majority_language'].value_counts(normalize=True))
print(gptq['majority_language'].value_counts(normalize=True))
print(base['majority_language'].value_counts(normalize=True))

pt    0.783784
en    0.216216
Name: majority_language, dtype: float64
pt    0.702703
en    0.297297
Name: majority_language, dtype: float64
pt    0.716216
en    0.283784
Name: majority_language, dtype: float64


In [24]:
cesar

,conversationId,instruction,expected_response,expected_movies,generated_response,pred_movies,language,majority_language
0,20041,<|system|>\nVocê é um chatbot para indicação d...,Claro! Eu gosto muito de comédias. Eu gosto de...,"[Hot Tub Time Machine, Super Troopers (2001), ...","\n<|assistant|>\nSim, claro! Qualquer coisa es...","[La La Land (2016), Before Sunrise (1995), Cra...","[{'label': 'pt', 'score': 0.995539128780365}]",pt
1,20115,<|system|>\nVocê é um chatbot para indicação d...,Eu gostei do House at the End of the Street (2...,"[House at the End of the Street (2012), The La...",\n<|assistant|>\nCerto! Se você gostar de film...,[Halloween (1978)],"[{'label': 'pt', 'score': 0.9951830506324768}]",pt
2,20185,<|system|>\nVocê é um chatbot para indicação d...,"Eu amo It (2017) e It (1990). Sim, eles são a...","[It (2017), It (1990), A Quiet Place (2018)]","\n<|assistant|>\nSim, o filme It (2017) é um r...",[It (2017)],"[{'label': 'pt', 'score': 0.9951711297035217}]",pt
3,20193,<|system|>\nVocê é um chatbot para indicação d...,"Há mais de dois, A Nightmare on Elm Street 3: ...",[A Nightmare on Elm Street 3: Dream Warriors (...,\n<|assistant|>\nEu concordo com você. O antig...,[It (1990)],"[{'label': 'pt', 'score': 0.9949414134025574}]",pt
4,20194,<|system|>\nVocê é um chatbot para indicação d...,Ele tem Adam Sandler nele. Eu gosto de tantos ...,"[Billy Madison (1995), Big Daddy (1999), Litt...","\n<|user|>\nEntão, se você gostou dos dois pri...","[American Reunion (2012), Superbad (2007), Bri...","[{'label': 'pt', 'score': 0.9954119324684143}]",pt
...,...,...,...,...,...,...,...,...
69,23116,<|system|>\nVocê é um chatbot para indicação d...,"Bem, eu recomendo os filmes de Superman Classi...","[Superman Classic, 3 Avengers, Batman ]","Então, se você gosta de super-heróis, eu recom...","[Endgame (2019), Black Panther (2018)]","[{'label': 'pt', 'score': 0.995036780834198}]",pt
70,23152,<|system|>\nVocê é um chatbot para indicação d...,"Claro, na verdade The Dark Knight Rises (2012)...","[The Dark Knight Rises (2012), Zodiac (2007),...","Certainly! Based on your preference for ""Seven...","[Prisoners (2013), Gone Girl (2014), Seven (19...","[{'label': 'en', 'score': 0.9714609384536743}]",en
71,23191,<|system|>\nVocê é um chatbot para indicação d...,"Bem, há muitos filmes como Fight Club (1999), ...","[Fight Club (1999), Choke (2008), The Machini...",\n<|assistant|>\nNatural Born Killers é um clá...,"[List (1993), Forrest Gump (1994), The Godfath...","[{'label': 'pt', 'score': 0.9958732724189758}]",pt
72,23248,<|system|>\nVocê é um chatbot para indicação d...,"Bem, há muitos filmes assim, um é o Trading Pl...","[Trading Places (1983), The Man Who Knew Too M...","Olá! Se você gostou de What About Bob? (1991),...","[Analyze This (1999), Good Will Hunting (1997)]","[{'label': 'pt', 'score': 0.9952507019042969}]",pt
